In [2]:
import pandas as pd

In [3]:
# test set

In [4]:
df = pd.read_csv('/mnt/green-efs/oleksandr.korniienko/data/wi_locness.dev.gold.bea18.csv')
df

,src,task
0,"It 's difficult answer at the question "" what ...",gec
1,When I was younger I used to say that I wanted...,gec
2,I would like to study Psychology because one d...,gec
3,It 's difficult because I 'll have to study ha...,gec
4,"Maybe I 'll change my mind , maybe not .",gec
...,...,...
4379,How much violence is there ?,gec
4380,The most violent cartoons are ' Teenage Mutant...,gec
4381,As for the shows containing sinister combat vi...,gec
4382,The cause for television violence is to add te...,gec


In [5]:
samples = []
instruction = "Fix all grammatical errors ."
for _, row in df.iterrows():
    sample =   {
        "instruction": instruction,
        "input": row.src,
        "output": "",
        "history": []
    }
    samples.append(sample)
samples[0]

{'instruction': 'Fix all grammatical errors .',
 'input': 'It \'s difficult answer at the question " what are you going to do in the future ? " if the only one who has to know it is in two minds .',
 'output': '',
 'history': []}

In [6]:
import json
with open('dataset.json', 'w') as f:
    json.dump(samples, f, indent=2)


In [26]:
import spacy
NLP = spacy.load('en_core_web_sm')

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

def tokenize(txt):
    doc = NLP(txt)
    return " ".join([t.text for t in doc])



[nltk_data] Downloading package punkt to
[nltk_data]     /home/oleksandr.korniienko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
# Gemma-7B-chat-gec  LLaMA2-7B-chat-gec  LLaMA2-7B-gec  Mistral-7B-Instruct-gec  phi-2-gec
model_name = 'Gemma-7B-chat-gec'
model_name = 'LLaMA2-7B-chat-gec'


data_path = f'/home/oleksandr.korniienko/LLaMA-Factory/saves/{model_name}/lora-all/predict/generated_predictions.jsonl'
output_path = f'/home/oleksandr.korniienko/LLaMA-Factory/saves/{model_name}/lora-all/predict/output.csv'

import json
with open(data_path, 'r') as f:
    data = [json.loads(l) for l in f.readlines()]

df_output = pd.DataFrame(data)

df = pd.read_csv('/mnt/green-efs/oleksandr.korniienko/data/nucle.test.csv')

df['output'] = df_output['predict']

df['output'] = df.output.map(tokenize)

df.to_csv(output_path, index=False)

output_path

'/home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-chat-gec/lora-all/predict/output.csv'

In [36]:
cmd = f's2s_evaluate --eval /mnt/green-efs/kostiantyn.omelianchuk/data/evalsets/nucle14-2a.m2 --pred {output_path}  --metrics max_match wed --print-format yaml'

cmd

's2s_evaluate --eval /mnt/green-efs/kostiantyn.omelianchuk/data/evalsets/nucle14-2a.m2 --pred /home/oleksandr.korniienko/LLaMA-Factory/saves/LLaMA2-7B-chat-gec/lora-all/predict/output.csv  --metrics max_match wed --print-format yaml'

In [7]:
# [INST] <<SYS>>\nRewrite this text to make it grammatically correct .\n<</SYS>>\n\n{src} [/INST]

template = """
Rewrite this text to make it grammatically correct .

{text}"""

df = pd.read_json('troy-llama7-gec-20k-greco.json')
df['prompt'] = df.src.map(lambda x: template.format(text=x))
df['chosen'] = df['choosen'].map(tokenize)
df['rejected'] = df['rejected'].map(tokenize)


FileNotFoundError: File troy-llama7-gec-20k-greco.json does not exist